# One Hidden Layer NN
---
We will build a shallow dense neural network with one hidden layer, and and the following structure is used for illustration purpose.

<img src='images/1-hidden-nn.png'>

Where in the graph above, we have a input vector $x = (x_1, x_2)$, containing 2 features and 4 hidden units $a1, a2, a3$ and $a4$, and output one value $y_1 \in [0, 1]$ (consider this a binary classification task with a prediction of probability)

In each hidden unit, take $a_1$ as example, a linear operation followed by an activation function is conducted. So given input $x = (x_1, x_2)$, inside node $a_1$, we have:

$$z_1 = w_{11}*x_1 + w_{12}*x_2$$
$$a_1 = activation(z_1)$$

Here $w_{11}$ denotes weight 1 of node 1, $w_{12}$ denotes weight 2 of node 1. Same for node $a_2$, it would have:

$$z_2 = w_{21}*x_1 + w_{22}*x_2$$
$$a_2 = activation(z_2)$$

And $a_3$ and $a_4$ so on ...

# Vectorization of One Input
---
